In [1]:
import os
import sys
import tensorflow as tf
import tensorflow.compat.v1 as tf_v1
from donkeycar import Vehicle
from donkeycar.parts.dgym import DonkeyGymEnv
from donkeycar.parts.datastore import TubHandler
from donkeycar.parts.controller import LocalWebController

/home/rajk/miniconda3/envs/donkey/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rajk/miniconda3/envs/donkey/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rajk/miniconda3/envs/donkey/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rajk/miniconda3/envs/donkey/lib/python3.7/s

using donkey v3.1.1 ...


In [2]:
SRC_DIR = os.path.dirname(os.path.abspath("."))
sys.path.insert(0, SRC_DIR)

In [3]:
from src.model import DonkeyNet
from src.utils import crop_img

In [4]:
class DriveMode:
    @staticmethod
    def run(user_steering, user_throttle, pilot_steering, pilot_throttle, mode):
        auto_mode = (mode != "user")
        if auto_mode:
            return pilot_steering, pilot_throttle, auto_mode
        return user_steering, user_throttle, auto_mode

In [5]:
class DonkeyNetCtr:
    def __init__(self, sess, input_shape, model_path, throttle=0.15):
        self.sess = sess
        self.throttle = throttle
        self.input_shape = input_shape
        self.model = DonkeyNet(version=1, input_shape=input_shape)
        sess.run(tf.global_variables_initializer())
        self.model.restore_model(sess, os.path.join(model_path, "model.chkpt"))
        
    def run(self, img_array):
        img_array = [crop_img(img_array, crop_dim=(slice(20, -1),))]
        steering = self.model.predict(self.sess, img_array)[0][0]
        return steering, self.throttle
    
    def shutdown():
        self.sess.close()

In [6]:
class RandomController:
    def __init__(self, env):
        self.env = env
        
    def run(self):
        steering, throttle = self.env.action_space.sample()
        return steering, throttle

In [7]:
os.environ["CUDA_VISIBLE_DEVICES"]="-1"
SIM_PATH = r"/home/rajk/Machine_Learning/DonkeySimLinux/donkey_sim.x86_64"
RECORD_PATH = r"/home/rajk/LogDir/"
MODEL_PATH = r"/home/rajk/Machine_Learning/DonkeyCar Project/models/DonkeyNetV1Model_BigNoFlip/"
PORT = 9090
HEADLESS = 0

In [8]:
car = Vehicle()

In [9]:
GPU_OPTIONS = tf_v1.GPUOptions(per_process_gpu_memory_fraction=0.75)
TF_CONFIG = tf_v1.ConfigProto(gpu_options=GPU_OPTIONS, allow_soft_placement=True)
sess = tf_v1.Session(config=TF_CONFIG)

In [10]:
donkey_env = DonkeyGymEnv(SIM_PATH, port=PORT, headless=HEADLESS)
cam = {"part": donkey_env, 
       "inputs": ["steering", "throttle"],
       "outputs": ["cam/image_array"], 
       "threaded": True}

ctr = {"part": LocalWebController(), 
       "inputs": ["cam/image_array"],
       "outputs": ["user/steering", "user/throttle", "user/mode", "recording"], 
       "threaded": True}

random_ctr = {"part": RandomController(donkey_env.env),              
              "outputs": ["random/steering", "random/throttle"],
              "run_condition": "drive/auto"}

input_shape = list(donkey_env.env.observation_space.shape)
input_shape[0] = 99
throttle=0.35
donkeynet_ctr = {"part": DonkeyNetCtr(sess, input_shape=input_shape, model_path=MODEL_PATH, throttle=throttle),
             "inputs": ["cam/image_array"],
             "outputs": ["random/steering", "random/throttle"],
             "run_condition": "drive/auto"}

drive_mode = {"part": DriveMode(),
              "inputs": ["user/steering", "user/throttle", "random/steering", "random/throttle", "user/mode"],
              "outputs": ["steering", "throttle", "drive/auto"]}

tub_handler = TubHandler(path=RECORD_PATH)
tub_inputs = ["cam/image_array", "user/angle", "user/throttle"]
tub_input_types = ["image_array", "float", "float"]
tub_writer = {"part": tub_handler.new_tub_writer(inputs=tub_inputs, types=tub_input_types, user_meta=[]),
              "inputs": tub_inputs,
              "outputs": ["tub/num_records"], 
              "run_condition": "recording"}

/home/rajk/miniconda3/envs/donkey/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
waiting for sim to start..


starting DonkeyGym env
donkey subprocess started
binding to ('0.0.0.0', 9090)


waiting for sim to start..
waiting for sim to start..
waiting for sim to start..


got a new client ('127.0.0.1', 44264)
connection dropped
got a new client ('127.0.0.1', 44266)
Starting Donkey Server...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model DonkeyNetV1Model with 14 built!

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /home/rajk/Machine_Learning/DonkeyCar Project/models/DonkeyNetV1Model_BigNoFlip/model.chkpt
Model restored from '/home/rajk/Machine_Learning/DonkeyCar Project/models/DonkeyNetV1Model_BigNoFlip/model.chkpt'!
Tub does NOT exist. Creating new tub...
New tub created at: /home/rajk/LogDir/tub_24_20-02-14


In [11]:
for part in (cam, ctr, donkeynet_ctr, drive_mode, tub_writer):
    car.add(**part)

Adding part DonkeyGymEnv.
Adding part LocalWebController.
Adding part DonkeyNetCtr.
Adding part DriveMode.
Adding part TubWriter.


In [12]:
car.start()

Web controller port: 8887
Starting vehicle...
fps 12.472032142087839
fps 19.98883013246906
fps 19.997573193370002
fps 19.990087655947963
fps 20.05207041027081
fps 19.97695814892462
fps 20.01256873811959
fps 19.984271983666876
fps 19.979806329014313
fps 20.05543294767437
fps 19.98918260415199
fps 19.991958038424865
fps 20.002366345925534
fps 19.970599542128387
fps 19.994673240690435
fps 20.05010346040468
fps 20.005903078472596
fps 19.98803187769469
fps 19.984295788069375
fps 19.995261361648208
fps 20.059082485443394
fps 19.972106788838467
fps 19.997699048623833
fps 19.987866138276747
fps 19.959608753400538
fps 20.087034634405352
fps 19.970240118780513
fps 19.99810904263346
fps 19.980464009546484
fps 19.99127387423302
fps 20.00776115116905
fps 20.036933312135407
fps 19.998042298276978
fps 19.99668557743706
fps 19.988277634304794
fps 19.96981985709425
fps 20.047803423786632
fps 20.005424062787103
fps 19.979476077407092
fps 19.989003508878614
fps 19.991211939674766
fps 20.05860283832092
fp